In [ ]:
# Part 1. Network Friendly Recommendations Project
import numpy as np
import random
import itertools
import matplotlib.pyplot as plt

#CONTENT CATALOGUE
#number of content items
K=50
#K = 2200

#content relativity matrix
u_ij = np.random.rand(K, K)

#percentage of cached items
C=0.2*K
C=int(C)

#cache
cache=[0]*C

#content relevance threshold
u_min=0.4

#USER MODEL
#number of recomendations
N=2
#probability of ending viewing session
q=0.01
#probability of picking one item with equal prob. in case all are relevant
alpha=0.9
#probability when the user chooses from K items
p_k=1/K

# equates each action set to an integer
actions=[]

rewards=[]

# function that initializes the cache
def initialize_cache():
   items_in_cache = random.sample(list(range(K)), C)
   for i in range(len(items_in_cache)):
    cache[i]=items_in_cache[i]

# function that initializes the relativity matrix u_ij
def initialize_relativity_matrix():
  # set the diagonal elements of the array to 0
  for i in range(K):
    for j in range(0, i):
        u_ij[i][j] = u_ij[j][i]
    u_ij[i][i] = 0

# function used to determine the cost of the item
def item_cost(item):
  if item in cache:
    return 0
  else:
    return -1

#check if all items are above u_min
def all_bigger_than_u_min(current_item,recommended_items):
  for i in recommended_items:
    relevance=u_ij[i,current_item]
    if(relevance<u_min):
      return False
  return True

#simulate user behaviour
def user_sim(current_state,recommended_items):
  done=False # Flag that indicates if a session is ended
  new_state=-1
  reward=0

  end_session=np.random.rand()
  #check if the user quits watching stupid memes and decides to do something better with their lives
  if(end_session<q):
    done=True
  else: #user continues procrastinating instead of doing RL project
    #check if all recomended items are relevant
    if all_bigger_than_u_min(current_state,recommended_items):
        p=np.random.rand()
        # if p < a user picks one of the recommended items
        if (p<alpha):
          new_state = random.choice(recommended_items)
          # check if the current state is the same as the new one
          while new_state == current_state:
            new_state = random.choice(recommended_items)
        # if not they pick any item from the catalogue
        else:
          possible_states=list(range(0,K))
          possible_states.remove(current_state)
          new_state = random.choice(possible_states)

    else:
        possible_states=list(range(0,K))
        possible_states.remove(current_state)
        new_state = random.choice(possible_states)


    reward=item_cost(new_state)
  #print(new_state , reward , done)
  return new_state , reward , done

def initialize_actions():
  items = list(range(K))
  recommended_combinations = list(itertools.combinations(items, N))

  for item_set in recommended_combinations:
    actions.append(item_set)











initialize_relativity_matrix()
initialize_cache()
initialize_actions()
# Print the content catalogue
#print("Content Catalogue:")
#print(u_ij)
print("\nCached Items:", cache)
#print(actions)

# Initialize Q-table
states = K  # Number of possible states
K_minus=K-1 #it cant transition from e.g state 9 again to state 9
actions_num = len(actions) # Number of possible actions
Q_table = np.zeros((states, actions_num))  # init Q-table


#encode each set of actions to a number in order to be compatible with the matrix

gamma = 0.5 # discount factor

# Q-learning algorithm
def Q_learning_algorithm(max_episodes, gamma):
  learning_rate = 0.1
  # for tests:
  # learning_rate = 0.3
  # learning_rate = 0.7
  #gamma = 0.5 # discount factor
  exploration_rate = 1
  max_exploration_rate=1
  min_exploration_rate=0.01
  exploration_decay_rate=0.01


  for i in range(max_episodes):
    state = random.choice(list(range(0,K))) # initialize start state
    avg_reward = 0
    num_of_watched_items = 0
    while True:
      # Choose an action using epsilon-greedy policy
      if np.random.random() < exploration_rate:
        action = np.random.randint(len(actions))
        while state in actions[action]:
          action = np.random.randint(len(actions))
      else:
        action = np.argmax(Q_table[state])
        while state in actions[action]:
          Q_table[state,action]=float('-inf')
          action = np.argmax(Q_table[state])

      next_state,reward,done=user_sim(state,actions[action])


      # Update the Q-table
      Q_table[state, action] = (1 - learning_rate) * Q_table[state, action] + learning_rate * (reward + gamma * np.max(Q_table[next_state, :]))

      # update values
      num_of_watched_items+=1
      state=next_state
      avg_reward = avg_reward + reward

      exploration_rate = min_exploration_rate + \
      (max_exploration_rate-min_exploration_rate)*np.exp(-exploration_decay_rate*i)
      # Check if session ends
      if done:
        rewards.append(avg_reward/num_of_watched_items)
        break




# run the algorithm
gamma = 0.5
dicrete_rewards=[]
gamma0 = 0.1
dicrete_rewards1=[]
gamma2 = 0.9
dicrete_rewards2=[]

# run it for 3 gamma values
for i in range(20):
  avg_cost = Q_learning_algorithm(1000, gamma)
  dicrete_rewards.append(sum(rewards)/len(rewards))
  print(i*5+5,'% done, cost:',sum(rewards)/len(rewards)) # used to observe how fast it is running
  rewards.clear()

for i in range(20):
  Q_table = np.zeros((states, actions_num))
  avg_cost = Q_learning_algorithm(1000, gamma0)
  dicrete_rewards1.append(sum(rewards)/len(rewards))
  print(i*5+5,'% done, cost:',sum(rewards)/len(rewards)) # used to observe how fast it is running
  rewards.clear()

for i in range(20):
  Q_table = np.zeros((states, actions_num))
  avg_cost = Q_learning_algorithm(1000, gamma2)
  dicrete_rewards2.append(sum(rewards)/len(rewards))
  print(i*5+5,'% done, cost:',sum(rewards)/len(rewards)) # used to observe how fast it is running
  rewards.clear()


# Create the plot
plt.plot(dicrete_rewards)
plt.plot(dicrete_rewards, label='gamma = 0.5')
plt.plot(dicrete_rewards1, color='r', label='gamma = 0.1')
plt.plot(dicrete_rewards2, color='g', label='gamma = 0.9')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('')

plt.title("cost")


# Display the plot
plt.show()






Cached Items: [47, 4, 19, 43, 9, 7, 6, 1, 18, 8]
5 % done, cost: -0.6705140862449153
10 % done, cost: -0.6314774228457644
15 % done, cost: -0.5691809816236134
